# Investigando algo genético para planear torneos *Double Round Robin*

En torneos Double Round Robin todos los equipos se enfrentan 2 veces, una vez en casa y otra de visita.

Como conocemos todas las partidas de antemano, la planeación de un torneo se reduce a ordenar las partidas de una forma válida/óptima. Para que una asignacion sea valida cada equipo debe jugar exactamente una vez por jornada.

Si hay n equipos hay
- n(n-1) juegos en el torneo
- n/2 juegos por jornada
- y 2(n-1) jornadas.

Representamos una partida con la pareja X,Y, que indica que juega X vs. Y en casa de X.

Codigo en ingles. Jornada->week.

In [11]:
import numpy as np
import random,time,json
from tqdm import tqdm
import os

In [12]:
# Given the number of teams, returns the list of games that must be played
def games(nTeams):
    games=[]
    for i in range(1,nTeams+1):
        for j in range(1,nTeams+1):
            if i==j:continue
            games.append((i,j))
    assert len(games)==nTeams*(nTeams-1)
    return games

In [13]:
# Returns a random assignment/tournament by shuffling the list of games
def randomAssignment(nTeams):
    p=games(nTeams)
    random.shuffle(p)
    return p

In [14]:
# Returns a random assignment of teams to cities
def randomCityTeams(nTeams,nCities):
    cities={i:[] for i in range(1,nCities+1)} # city:[team1,team2]
    randTeams=list(range(1,nTeams+1))
    random.shuffle(randTeams)
    while len(randTeams)>0:
        city=random.choice(range(1,nCities+1))
        cities[city].append(randTeams.pop())
    return cities

In [15]:
# Given a cityTeams dict (city:[list of teams]), returns a dict (team:city)
def teamToCity(cityTeams):
    out={}
    for city,teams in cityTeams.items():
        for team in teams:
            out[team]=city
    return out

In [16]:
def randomCityDistances(nCities,low=0,high=100):
    r=np.random.randint(low,high,size=(nCities,nCities))
    return (r+r.T)/2

In [17]:
randomCityDistances(4)

array([[48. , 83. , 59. , 59. ],
       [83. , 48. , 66. , 51.5],
       [59. , 66. , 82. , 31. ],
       [59. , 51.5, 31. , 36. ]])

In [18]:
randomCityTeams(4,3)

{1: [4, 2], 2: [1, 3], 3: []}

In [19]:
randomAssignment(4)

[(4, 3),
 (1, 4),
 (4, 2),
 (2, 4),
 (4, 1),
 (2, 3),
 (1, 2),
 (3, 4),
 (1, 3),
 (3, 1),
 (3, 2),
 (2, 1)]

In [20]:
def lastWeek(assignment,nTeams):
    week=[]
    for i in range(0,len(assignment),nTeams//2):
        week=assignment[i:i+nTeams//2]
    return week

In [21]:
def approx(nTeams):
    current=[]
    gs=games(nTeams)
    remaining=gs[:]
    random.shuffle(remaining)
    week=[]
    while len(current)!=nTeams*(nTeams-1):
        available=[]
        for game in remaining:
            if game[0] not in week and game[1] not in week:
                available.append(game)
        game=random.choice(available) if available else remaining.pop()
        current.append(game)
        week.append(game)
        week=week[-nTeams//2:]
        remaining.remove(game)
    assert len(set(current))==nTeams*(nTeams-1)
    return current   

In [23]:
def approxAssignment(nTeams):
    current=[]
    gs=games(nTeams)
    remaining=gs[:]
    while len(current)!=nTeams*(nTeams-1):
        available=[]
        inLastWeek=teamsInGames(lastWeek(current,nTeams))
        for game in remaining:
            if game[0] not in inLastWeek and game[1] not in inLastWeek:
                available.append(game)
        game=random.choice(available) if available else random.choice(remaining)
        current.append(game)
        remaining.remove(game)
    assert len(set(current))==nTeams*(nTeams-1)
    return current

In [24]:
# Used in conflicts(). Given a list of len 2 tuples, returns a list with all elements.
def teamsInGames(week):
    out=[]
    for game in week:
        out.append(game[0])
        out.append(game[1])
    return out

In [26]:
games(4)

[(1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3)]

In [27]:
r=randomAssignment(4)
r

[(4, 3),
 (1, 4),
 (2, 4),
 (3, 2),
 (2, 3),
 (4, 2),
 (3, 4),
 (1, 3),
 (3, 1),
 (1, 2),
 (4, 1),
 (2, 1)]

In [29]:
r[0]=(1,4)
r

[(1, 4),
 (1, 4),
 (2, 4),
 (3, 2),
 (2, 3),
 (4, 2),
 (3, 4),
 (1, 3),
 (3, 1),
 (1, 2),
 (4, 1),
 (2, 1)]

In [98]:
# Returns the conflicts in an assignment/tournament.
# Marks as 1 games in which a participating team plays more than once in that week.
def conflicts(assignment,nTeams):
    conflicts=[]
    n=nTeams 
    for i in range(0,len(assignment),n//2):
        week=assignment[i:i+n//2]
        #print(week)
        teamsInWeek=teamsInGames(week)
        for game in week:
            conflicts.append(0)
            #if assignment.count(game)!=1:
             #   conflicts[-1]=1
            #else:
            for team in game:
                if teamsInWeek.count(team)!=1:
                    conflicts[-1]=1
    return conflicts

In [156]:
# Returns the conflicts in an assignment/tournament.
# Marks as 1 games in which a participating team plays more than once in that week.
def nConflicts2(assignment,nTeams):
    c=0
    n=nTeams 
    for i in range(0,len(assignment),n//2):
        week=assignment[i:i+n//2]
        teamsInWeek=teamsInGames(week)
        print(week,teamsInWeek,set(teamsInWeek))
        print(min((len(teamsInWeek)-len(set(teamsInWeek)))*2,len(week)))
        c+=min((len(teamsInWeek)-len(set(teamsInWeek)))*2,len(week))
        print(c)
    return c

In [142]:
for i in range(100):
    p=randomAssignment(8)
    if nConflicts(p,8)!=nConflicts2(p,8):
        pass
        #print(i,nConflicts(p,8),nConflicts2(p,8))

In [171]:
nConflicts2(p,6)

[(2, 1), (4, 3), (3, 2)] [2, 1, 4, 3, 3, 2] {1, 2, 3, 4}
3
3
[(4, 2), (1, 4), (3, 1)] [4, 2, 1, 4, 3, 1] {1, 2, 3, 4}
3
6
[(3, 4), (1, 3), (1, 2)] [3, 4, 1, 3, 1, 2] {1, 2, 3, 4}
3
9
[(2, 4), (4, 1), (2, 3)] [2, 4, 4, 1, 2, 3] {1, 2, 3, 4}
3
12


12

In [174]:
a=conflicts(p,4)
n=4
for i in range(0,len(a),n//2):
        
        week=a[i:i+n//2]
        print(p[i:i+n//2],week,sum(week))

[(2, 1), (4, 3)] [0, 0] 0
[(3, 2), (4, 2)] [1, 1] 2
[(1, 4), (3, 1)] [1, 1] 2
[(3, 4), (1, 3)] [1, 1] 2
[(1, 2), (2, 4)] [1, 1] 2
[(4, 1), (2, 3)] [0, 0] 0


In [191]:
nConflicts(p,8)

50

In [ ]:
nConflicts([[5,2],[8,6],[1,3],[5,1],[8,7],[4,2],[6,1],[2,6],[6,8],[6,7],[3,8],[7,4],[5,6],[7,8],[5,8],[1,2],[5,3],[2,5],[5,7],[1,7],[4,5],[5,4],[7,2],[4,1],[3,6],[6,4],[6,3],[1,8],[2,4],[7,1],[7,6],[6,5],[4,8],[3,5],[4,6],[1,5],[2,3],[7,5],[3,2],[8,2],[8,4],[1,6],[3,4],[2,7],[8,3],[2,1],[4,7],[3,7],[1,4],[8,5],[7,3],[3,1],[2,8],[8,1],[4,3],[6,2]],8,X).

In [189]:
nConflicts([[2,5],[7,5],[4,1],[4,2],[1,3],[7,3],[7,8],[6,7],[5,8],[5,6],[1,4],[1,2],[7,6],[6,3],[4,8],[8,3],[3,4],[6,1],[7,2],[2,1],[4,3],[6,2],[5,2],[2,7],[8,5],[3,6],[6,4],[3,7],[8,2],[6,8],[7,1],[1,8],[3,2],[7,4],[5,7],[3,1],[8,7],[8,4],[5,1],[2,3],[1,5],[3,8],[4,7],[8,1],[5,4],[4,6],[8,6],[1,7],[2,6],[2,4],[2,8],[3,5],[4,5],[6,5],[1,6],[5,3]],8)

48

In [188]:
print('[',end='')
for t in p:
    print('[{},{}],'.format(t[0],t[1]),end='')

[[4,5],[8,2],[1,6],[3,7],[2,8],[4,3],[6,1],[7,5],[6,8],[5,7],[3,1],[4,2],[2,4],[8,1],[7,3],[5,6],[5,4],[1,8],[2,6],[4,1],[2,5],[6,7],[4,8],[1,3],[3,4],[7,2],[5,8],[8,3],[6,2],[1,5],[3,8],[7,4],[7,6],[8,5],[3,2],[1,4],[2,3],[8,4],[7,1],[6,5],[5,2],[8,7],[6,3],[4,6],[4,7],[8,6],[2,1],[5,3],[7,8],[6,4],[1,2],[3,5],[2,7],[5,1],[3,6],[1,7],

In [190]:
p=approx(8)
p

[(4, 7),
 (7, 8),
 (3, 6),
 (5, 8),
 (1, 3),
 (3, 4),
 (8, 1),
 (2, 7),
 (3, 8),
 (3, 7),
 (3, 2),
 (1, 2),
 (4, 1),
 (6, 4),
 (3, 5),
 (7, 1),
 (1, 6),
 (7, 6),
 (5, 3),
 (7, 4),
 (4, 8),
 (1, 4),
 (5, 1),
 (1, 8),
 (7, 3),
 (2, 8),
 (8, 6),
 (7, 2),
 (5, 2),
 (2, 1),
 (8, 3),
 (6, 5),
 (8, 4),
 (4, 6),
 (8, 2),
 (6, 8),
 (8, 5),
 (2, 6),
 (2, 3),
 (5, 6),
 (6, 2),
 (3, 1),
 (4, 2),
 (6, 3),
 (1, 5),
 (8, 7),
 (6, 1),
 (7, 5),
 (5, 4),
 (4, 3),
 (4, 5),
 (1, 7),
 (6, 7),
 (2, 5),
 (5, 7),
 (2, 4)]

In [52]:
p[0]=(4,2)

In [54]:
p

[(4, 2),
 (3, 1),
 (4, 2),
 (1, 3),
 (2, 1),
 (4, 1),
 (2, 4),
 (1, 4),
 (4, 3),
 (2, 3),
 (3, 2),
 (1, 2)]

In [53]:
conflicts(p,4)

[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [55]:
# Returns the number of conflicts in an assignment. 
def nConflicts(assignment,nTeams):
    return sum(conflicts(assignment,nTeams))

## Min conflits (base line)

Para comparar el algo genetico con algo, intente implementar Min Conflicts (ver busqueda local en el lib) con swaps.

In [14]:
# Given the index of a conflict, return the game swap that results least nConflicts.
def minSwap(assignment,conflict,nTeams):
    bestN,bestOthers=None,[]
    for i in range(len(assignment)):
        assignment[conflict],assignment[i]=assignment[i],assignment[conflict]
        n=nConflicts(assignment,nTeams)
        if not bestN or n==bestN:
            bestN=n
            bestOthers.append(i)
        elif bestN and n<bestN:
            bestN=n
            bestOthers=[i]       
        assignment[conflict],assignment[i]=assignment[i],assignment[conflict]
    return random.choice(bestOthers)

In [15]:
# Returns the index of a random conflict in assignment.
def randomConflict(assignment,nTeams):
    indices=np.argwhere(conflicts(assignment,nTeams))
    return indices[np.random.randint(0,indices.shape[0])][0]

In [16]:
# Min Conflicts implementation.
# Returns final state of current and wether it was solved (nConflicts==0)
def minConflicts(nTeams,maxSteps,iniApprox=True,debug=False,autoEndK=100):
    current=approxAssignment(nTeams) if iniApprox else randomAssignment(nTeams)
    n=nConflicts(current,nTeams)
    lastK=[]
    for i in range(maxSteps):
        n=nConflicts(current,nTeams)
        lastK.append(n)
        lastK=lastK[-autoEndK:]
        if debug:print(n)
        if n==0 or (lastK and lastK.count(n)==autoEndK):break
        conflict=randomConflict(current,nTeams)
        other=minSwap(current,conflict,nTeams)
        current[conflict],current[other]=current[other],current[conflict]
    return current,i#i!=maxSteps-1

In [17]:
def kMinConflictsSolutions(n,k):
    solutions=[]
    avrgTime,avrgSteps,avrgResets=0,0,0
    maxSteps=n*15
    while len(solutions)<k:
        start=time.time()
        sol,steps=minConflicts(n,maxSteps,debug=False)
        t=time.time()-start
        if nConflicts(sol,n)==0:
            solutions.append(tuple(sol))
            avrgTime+=t
            avrgSteps+=steps
        else:
            avrgResets+=1
    return solutions,list(set(solutions)),avrgTime/k,avrgSteps/k,avrgResets/k,maxSteps

In [482]:
for n in range(20,32,2):
    sols,unique,t,steps,resets,maxSteps=kMinConflictsSolutions(n,100)
    with open("minConflictsSols/"+str(n)+".json","w") as f:
        f.write(json.dumps(sols))
    print(n,len(unique),t,steps,resets,maxSteps)

KeyboardInterrupt: 

In [483]:
minConflicts(32,1000,debug=True)

88
87
85
83
82
78
76
75
73
71
70
68
66
64
63
63
63
62
60
60
60
59
57
56
55
55
54
54
53
53
53
53
52
51
51
51
49
49
49
47
47
46
45
45
45
45
45
43
43
43
43
43
40
40
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
37
36
36
36
34
34
34
34
33
33
33
33
30
30
30
30
28
28
28
28
28
28
28
28
28
28
28
28
28
28
26
26
26
26
24
24
24
24
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
21
20
20
18
18
18
18
16
16
16
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
13
12
12
12
12
12
12
10
10
10
10
10
10
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
7
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


([(30, 11),
  (32, 9),
  (3, 7),
  (13, 2),
  (20, 29),
  (8, 22),
  (19, 5),
  (10, 24),
  (27, 14),
  (4, 1),
  (21, 18),
  (25, 17),
  (23, 15),
  (6, 28),
  (16, 12),
  (31, 26),
  (30, 18),
  (24, 20),
  (7, 17),
  (3, 5),
  (27, 19),
  (11, 22),
  (8, 2),
  (32, 28),
  (23, 14),
  (9, 10),
  (29, 31),
  (26, 15),
  (4, 6),
  (16, 25),
  (12, 13),
  (1, 21),
  (20, 8),
  (11, 9),
  (17, 12),
  (13, 3),
  (24, 18),
  (31, 25),
  (19, 16),
  (5, 1),
  (32, 30),
  (4, 23),
  (21, 7),
  (2, 15),
  (26, 27),
  (28, 10),
  (22, 6),
  (14, 29),
  (31, 3),
  (10, 30),
  (22, 24),
  (14, 16),
  (29, 25),
  (26, 17),
  (11, 5),
  (6, 27),
  (9, 8),
  (7, 2),
  (13, 32),
  (21, 19),
  (18, 23),
  (12, 15),
  (28, 4),
  (20, 1),
  (30, 31),
  (2, 19),
  (4, 10),
  (25, 3),
  (16, 8),
  (15, 22),
  (11, 14),
  (18, 5),
  (28, 13),
  (20, 7),
  (23, 12),
  (1, 27),
  (6, 21),
  (29, 26),
  (24, 9),
  (32, 17),
  (17, 7),
  (14, 21),
  (13, 9),
  (3, 20),
  (32, 15),
  (29, 2),
  (12, 1),
  (16,

In [ ]:
approx

In [185]:
# Average nConflicts per nTeams of randomAssignment and 
n=6
s=0
print("n,avrRandom,avrApprox,Max,%avr rndom,%avrApprox")
for n in range(4,42,2):
    a=0
    s=0
    for i in range(100):
        s+=nConflicts(randomAssignment(n),n)
        a+=nConflicts(approxAssignment(n),n)
    print(','.join(map(str,[n,s/100,a/100,n*(n-1),(s/100)/(n*(n-1)),(a/100)/(n*(n-1))])))

n,avrRandom,avrApprox,Max,%avr rndom,%avrApprox
4,9.88,0.0,12,0.8233333333333334,0.0
6,24.97,4.78,30,0.8323333333333333,0.15933333333333335
8,47.08,10.41,56,0.8407142857142856,0.18589285714285714
10,76.84,15.0,90,0.8537777777777779,0.16666666666666666
12,112.48,22.09,132,0.8521212121212122,0.16734848484848486
14,155.78,29.51,182,0.855934065934066,0.16214285714285714
16,204.21,36.08,240,0.850875,0.15033333333333332
18,261.62,43.22,306,0.854967320261438,0.14124183006535948
20,325.88,51.92,380,0.857578947368421,0.13663157894736844
22,395.0,59.52,462,0.854978354978355,0.12883116883116882
24,474.43,68.43,552,0.8594746376811594,0.12396739130434783


KeyboardInterrupt: 

## Genetic algo

Tenemos que definir operadores de mutacion y de reproduccion. Lo que se me ocurrio es que el de mutacion puede ser un swap aleatorio y el de cruzamiento.

In [62]:
def mutate(assignment):
    # Random swap
    i,j=np.random.choice(len(assignment),size=2,replace=False)
    assignment=assignment[:]
    assignment[i],assignment[j]=assignment[j],assignment[i]
    return assignment

In [63]:
def freeMutate(assignment,nTeams):
    a=assignment[:]
    k=random.randint(0,len(a)-1)
    a[k]=random.choice(games(nTeams))
    return a

In [64]:
def crossover(a,b):
    assert len(a)==len(b),"a and b must be same length"
    k=random.randint(0,len(a)) # Random crossover point
    #print(k)
    offspring1=a[:k]
    for game in b:
        if len(offspring1)==len(a):break
        if game not in offspring1:
            offspring1.append(game)
    
    #print(offspring1,len(offspring1))
    assert len(offspring1)==len(a)
    return offspring1

In [65]:
def freeCrossover(a,b):
    assert len(a)==len(b),"a and b must be same length"
    k=random.randint(0,len(a)) # Random crossover point
    offspring1=a[:k]+b[k:]
    offspring2=b[:k]+a[k:]
    assert len(offspring1)==len(a) and len(offspring2)==len(a)
    return offspring1,offspring2

In [66]:
def hamming(a,b):
    dist=0
    assert len(a)==len(b)
    for i in range(len(a)):
        if a[i]!=b[i]:
            dist+=1
    return dist

In [67]:
def hammingOfList(l):
    dist=0
    for i in range(len(l)):
        for j in range(i+1,len(l)):
            dist+=hamming(l[i],l[j])
    return dist

In [68]:
a=randomAssignment(5)
b=randomAssignment(5)

In [69]:
# Given an assignment, returns the sum of the time between games of pairs of teams
def timeBetweenPairs(assignment,nTeams):
    pairs=0
    time=0
    # For every pair of teams
    for i in range(1,nTeams+1):
        for j in range(i+1,nTeams+1):
            if i==j:continue
            if (i,j) in assignment and (j,i) in assignment:
                gameA=assignment.index((i,j))
                gameB=assignment.index((j,i))
                time+=abs(gameA-gameB)
                pairs+=1
                print(i,j,time,pairs)
      
    maxPossible=(pairs)**2
    out=time/maxPossible if maxPossible>0 else 0
    #print(pairs,time,maxPossible,out) 
    #assert out<1.1,(assignment,out)
    return out

In [70]:
r=[(1, 4), (4, 3), (1, 2), (3, 2), (3, 1), (2, 1), (4, 3), (1, 3), (4, 1), (3, 4), (2, 3), (2, 4)]
timeBetweenPairs(r,4)

1 2 3 1
1 3 6 2
1 4 14 3
2 3 21 4
3 4 29 5


1.16

In [71]:
r=randomAssignment(4)
r=freeMutate(r,4)
print(r)
timeBetweenPairs(r,4)

[(2, 1), (3, 1), (1, 4), (4, 2), (3, 4), (1, 2), (4, 3), (2, 4), (2, 3), (3, 2), (4, 1), (3, 4)]
1 2 5 1
1 4 13 2
2 3 14 3
2 4 18 4
3 4 20 5


0.8

In [72]:
def alterningScore(assignment, nTeams):
    teams={str(i):[] for i in range(1,nTeams+1)}
    
    for game in assignment:
        teams[str(game[0])].append(0) # 0 if local
        teams[str(game[1])].append(1) # 1 if visiting
        
    # Average of scores
    return np.array([aux_score(team) for team in teams.values()]).mean() 

def aux_score(arr):
    s=0
    previous=arr[0]
    for i in range(1,len(arr)):
        if arr[i]!=previous:
            s+=1
        previous=arr[i]
    return (s-1)/(len(arr)-2)

In [73]:
# Returns the proportion of weeks in which a city has a game
# given a tournament assignment and the teams in that city.
def cityOccupied(assignment,teamsInCity,nTeams):
    weeks=0
    nOccupied=0
    teamsInCity=set(teamsInCity)
    n=nTeams
    for i in range(0,len(assignment),n//2):
        weeks+=1
        week=assignment[i:i+n//2]
        occupied=False
        for game in week:
            if game[0] in teamsInCity:
                occupied=True
                break
        if occupied:
            nOccupied+=1
    return nOccupied/weeks          

In [74]:
# Returns the average of cityOccupied() for every city.
def citiesAlwaysOccupied(assignment,cityTeams,nTeams):
    return np.array([cityOccupied(assignment,teamsInCity,nTeams) for teamsInCity in cityTeams.values()]).mean()

In [75]:
r=randomAssignment(4)
r

[(1, 3),
 (1, 4),
 (3, 2),
 (4, 1),
 (1, 2),
 (3, 1),
 (3, 4),
 (2, 3),
 (4, 2),
 (2, 1),
 (2, 4),
 (4, 3)]

In [76]:
distances=randomCityDistances(3)
distances

array([[ 2. , 71.5, 61. ],
       [71.5, 50. , 74. ],
       [61. , 74. , 14. ]])

In [77]:
cityTeams=randomCityTeams(4,3)
cityTeams

{1: [4], 2: [1, 2, 3], 3: []}

In [78]:
teamCity=teamToCity(cityTeams)

In [79]:
distanceTraveled(r,1,teamCity,distances)

NameError: name 'distanceTraveled' is not defined

In [80]:
# Returns the sum of the distance travelled by each team in an assignment
def totalDistanceTravelled(assignment,cityTeams,cityDistances,nTeams):
    teamsCity=teamToCity(cityTeams)
    return sum([distanceTraveled(assignment,team,teamsCity,cityDistances) for team in range(1,nTeams+1)])  

In [81]:
# Assumes a Traveling Tournament Problem (i.e. teams don't return home)
def distanceTraveled(assignment,team,teamsToCity,cityDistances):
    distance=0
    city=teamsToCity[team]
    # Asume start at home
    for game in assignment:
        if team==game[1]: # If plays as visitor
            otherCity=teamsToCity[game[0]]
            distance+=2*cityDistances[city-1][otherCity-1]
            city=otherCity
        elif team==game[0]:
            otherCity=teamCity[team] # Home city
            distance+=2*cityDistances[city-1][otherCity-1]
            city=otherCity
    return distance

In [82]:
# Fitness function with which to eval individuals and determine their survaival.
# At first, we will only consider the number of conflicts.
# We can then add other soft requirements
def fitness(individual,nTeams,cityTeams,weights=[0.85,0.05,0.05,0.05]):
    return -nConflicts(individual,nTeams)
    """"
    assert abs(sum(weights)-1)<1e-5,sum(weights)
    # Normalizing nConflicts. To minimize.
    conflicts=1-(nConflicts(individual,nTeams)/len(individual))
    
    # timeBetweenPair of teams score. Normalized. To maximize.
    pairTimes=timeBetweenPairs(individual,nTeams)
    
    # alterningScore. Normalized. To maximize.
    alterning=alterningScore(individual,nTeams)
    
    # citiesAlwaysOccupied score. Normalized. To maximize.
    citiesAlways=citiesAlwaysOccupied(individual,cityTeams,nTeams)
    
    #print(conflicts,pairTimes,alterning,citiesAlways)
    
    return (conflicts*weights[0]+pairTimes*weights[1]+alterning*weights[2]+citiesAlways*weights[3])
    """

In [83]:
def evolve(seed,debug=False):
    random.seed(seed)
    np.random.seed(seed)
    # Genetic algo
    nTeams=4
    cityTeams=randomCityTeams(nTeams,nTeams-1)
    n=300 #size of population
    nElite=int(n*0.05) #proportion best of population to retain
    k=0.70 # can regulate "temperature"
    mutationRate=0.1 # rate at which to mutate individuals
    assert n%2==0,'N must be even to crossover'
    # Random inicialitation of population
    population=[approxAssignment(nTeams) for i in range(n)]
    scores=[fitness(ind,nTeams,cityTeams) for ind in population]
    gen=0
    didntImprove=0
    didntImproveMax=250
    lastMax=None
    while True:
        
        # SELECTION (Investigate other methods)
        # Elitist -> choose best n
        elite=sorted(population,key=lambda x:fitness(x,nTeams,cityTeams),reverse=True)[:nElite]

        # Tournament with remaining n-nElite
        fromTournament=[]
        for _ in range(n-nElite):
            a,b=np.random.choice(n,size=2,replace=False)
            best,worst=(a,b) if fitness(population[a],nTeams,cityTeams)>fitness(population[b],nTeams,cityTeams) else (b,a)
            if random.random()<k:
                fromTournament.append(population[best])
            else:
                fromTournament.append(population[worst])

        # CROSSOVER -> according to fitness
        population=[]
        for _ in range(n-nElite):
            a,b=np.random.choice(len(fromTournament),size=2,replace=False)
            population.append(freeCrossover(fromTournament[a],fromTournament[b])[0])

        # MUTATION
        for i in range(len(population)):
            if random.random()<mutationRate:
                population[i]=freeMutate(population[i],nTeams)

        #Add elite
        population=elite+population
        assert len(population)==n

        # LOGGING
        scores=np.array([fitness(ind,nTeams,cityTeams) for ind in population])
        conf=np.array([nConflicts(ind,nTeams) for ind in population])
        #print(','.join([str(int(i)) for i in scores]),np.array(scores).mean())
        if debug:
            print(gen,k,scores.mean(),scores.min(),scores.max(),(np.array(scores)==0).sum(),'didnt',didntImprove,(conf==0).sum())

        # STOPPING CRITIREA -> if 90% of population are valid sols. (Change later)
        if not lastMax or scores.max()>lastMax:
            lastMax=scores.max()
            didntImprove=0
        else:
            didntImprove+=1

        if didntImprove>=didntImproveMax:
            break
        gen+=1

    return gen,scores.mean(),scores.min(),scores.max(),(np.array(scores)==0).sum(),population

In [84]:
def uniqueSolutions(population):
    unique=set()
    for i in population:
        if nConflicts(i,nTeams)==0:
            unique.add(tuple(i))
    return unique

In [85]:
for i in range(10):
    gen,mean,_min,_max,valid,population=evolve(seed=i,debug=True)
    uniqueSols=len(uniqueSolutions(population))
    print(gen,mean,_min,_max,valid,uniqueSols)
    break

0 0.7 -3.8966666666666665 -12 0 65 didnt 0
1 0.7 -5.473333333333334 -12 0 26 didnt 0
2 0.7 -6.1466666666666665 -11 0 21 didnt 0
3 0.7 -6.3966666666666665 -12 0 20 didnt 0
4 0.7 -6.8566666666666665 -11 0 21 didnt 0
5 0.7 -6.84 -12 0 18 didnt 0
6 0.7 -6.633333333333334 -12 0 26 didnt 0
7 0.7 -6.696666666666666 -12 0 19 didnt 0
8 0.7 -7.05 -12 0 19 didnt 0
9 0.7 -7.06 -12 0 19 didnt 0
10 0.7 -6.786666666666667 -12 0 19 didnt 0
11 0.7 -6.463333333333333 -12 0 25 didnt 0
12 0.7 -6.9366666666666665 -12 0 18 didnt 0
13 0.7 -6.953333333333333 -12 0 23 didnt 0
14 0.7 -6.92 -12 0 22 didnt 0
15 0.7 -6.95 -11 0 19 didnt 0
16 0.7 -7.023333333333333 -12 0 18 didnt 0
17 0.7 -6.763333333333334 -12 0 20 didnt 0
18 0.7 -7.093333333333334 -12 0 16 didnt 0
19 0.7 -6.86 -12 0 19 didnt 0
20 0.7 -7.076666666666667 -12 0 18 didnt 0
21 0.7 -6.906666666666666 -12 0 22 didnt 0
22 0.7 -6.923333333333333 -12 0 19 didnt 0
23 0.7 -6.913333333333333 -12 0 19 didnt 0
24 0.7 -7.013333333333334 -12 0 17 didnt 0
25 0.7 -

213 0.7 -6.976666666666667 -12 0 21 didnt 0
214 0.7 -6.95 -11 0 19 didnt 0
215 0.7 -6.943333333333333 -12 0 19 didnt 0
216 0.7 -6.89 -12 0 20 didnt 0
217 0.7 -6.68 -12 0 19 didnt 0
218 0.7 -6.93 -12 0 19 didnt 0
219 0.7 -6.81 -12 0 20 didnt 0
220 0.7 -6.843333333333334 -12 0 21 didnt 0
221 0.7 -7.1066666666666665 -12 0 18 didnt 0
222 0.7 -7.206666666666667 -12 0 24 didnt 0
223 0.7 -7.116666666666666 -12 0 22 didnt 0
224 0.7 -7.133333333333334 -12 0 20 didnt 0
225 0.7 -7.1066666666666665 -12 0 23 didnt 0
226 0.7 -6.733333333333333 -12 0 20 didnt 0
227 0.7 -6.776666666666666 -12 0 19 didnt 0
228 0.7 -6.576666666666667 -12 0 20 didnt 0
229 0.7 -6.8966666666666665 -12 0 24 didnt 0
230 0.7 -7.153333333333333 -12 0 22 didnt 0
231 0.7 -7.136666666666667 -12 0 21 didnt 0
232 0.7 -7.383333333333334 -12 0 21 didnt 0
233 0.7 -7.133333333333334 -12 0 21 didnt 0
234 0.7 -6.963333333333333 -12 0 24 didnt 0
235 0.7 -6.866666666666666 -12 0 23 didnt 0
236 0.7 -6.843333333333334 -12 0 21 didnt 0
237 0.

425 0.7 -7.156666666666666 -12 0 20 didnt 0
426 0.7 -6.92 -12 0 18 didnt 0
427 0.7 -6.6866666666666665 -12 0 22 didnt 0
428 0.7 -6.8933333333333335 -12 0 20 didnt 0
429 0.7 -6.766666666666667 -12 0 19 didnt 0
430 0.7 -6.39 -12 0 27 didnt 0
431 0.7 -6.97 -12 0 18 didnt 0
432 0.7 -6.86 -12 0 22 didnt 0
433 0.7 -6.85 -12 0 26 didnt 0
434 0.7 -6.81 -12 0 25 didnt 0
435 0.7 -6.733333333333333 -12 0 22 didnt 0
436 0.7 -6.906666666666666 -12 0 21 didnt 0
437 0.7 -7.04 -12 0 20 didnt 0
438 0.7 -6.87 -12 0 20 didnt 0
439 0.7 -7.32 -12 0 17 didnt 0
440 0.7 -7.096666666666667 -12 0 20 didnt 0
441 0.7 -6.93 -12 0 18 didnt 0
442 0.7 -6.873333333333333 -12 0 22 didnt 0
443 0.7 -6.923333333333333 -12 0 18 didnt 0
444 0.7 -7.21 -12 0 15 didnt 0
445 0.7 -7.06 -12 0 20 didnt 0
446 0.7 -6.883333333333334 -12 0 18 didnt 0
447 0.7 -6.97 -12 0 18 didnt 0
448 0.7 -6.94 -12 0 19 didnt 0
449 0.7 -7.176666666666667 -12 0 20 didnt 0
450 0.7 -7.25 -12 0 20 didnt 0
451 0.7 -7.086666666666667 -12 0 25 didnt 0
452 0

639 0.7 -7.18 -12 0 18 didnt 0
640 0.7 -6.95 -12 0 15 didnt 0
641 0.7 -7.046666666666667 -12 0 18 didnt 0
642 0.7 -7.1033333333333335 -12 0 19 didnt 0
643 0.7 -7.216666666666667 -12 0 20 didnt 0
644 0.7 -7.133333333333334 -12 0 20 didnt 0
645 0.7 -7.1033333333333335 -11 0 19 didnt 0
646 0.7 -7.14 -12 0 17 didnt 0
647 0.7 -7.23 -12 0 18 didnt 0
648 0.7 -6.993333333333333 -12 0 27 didnt 0
649 0.7 -6.96 -12 0 26 didnt 0
650 0.7 -6.82 -12 0 24 didnt 0
651 0.7 -6.95 -12 0 22 didnt 0
652 0.7 -6.9 -12 0 19 didnt 0
653 0.7 -7.013333333333334 -12 0 18 didnt 0
654 0.7 -6.9 -12 0 23 didnt 0
655 0.7 -7.08 -12 0 17 didnt 0
656 0.7 -7.246666666666667 -12 0 19 didnt 0
657 0.7 -7.293333333333333 -12 0 16 didnt 0
658 0.7 -6.836666666666667 -12 0 25 didnt 0
659 0.7 -7.016666666666667 -12 0 22 didnt 0
660 0.7 -6.816666666666666 -12 0 24 didnt 0
661 0.7 -6.793333333333333 -12 0 19 didnt 0
662 0.7 -6.796666666666667 -12 0 20 didnt 0
663 0.7 -6.803333333333334 -12 0 22 didnt 0
664 0.7 -6.883333333333334 -12

848 0.7 -7.226666666666667 -12 0 19 didnt 0
849 0.7 -7.193333333333333 -12 0 19 didnt 0
850 0.7 -7.086666666666667 -12 0 21 didnt 0
851 0.7 -7.1066666666666665 -12 0 22 didnt 0
852 0.7 -7.22 -12 0 20 didnt 0
853 0.7 -7.62 -12 0 19 didnt 0
854 0.7 -7.483333333333333 -12 0 19 didnt 0
855 0.7 -7.45 -12 0 17 didnt 0
856 0.7 -7.5633333333333335 -12 0 18 didnt 0
857 0.7 -7.633333333333334 -12 0 17 didnt 0
858 0.7 -7.23 -12 0 19 didnt 0
859 0.7 -7.423333333333333 -12 0 17 didnt 0
860 0.7 -7.306666666666667 -12 0 22 didnt 0
861 0.7 -7.003333333333333 -12 0 24 didnt 0
862 0.7 -6.8566666666666665 -12 0 21 didnt 0
863 0.7 -7.01 -12 0 23 didnt 0
864 0.7 -6.793333333333333 -12 0 21 didnt 0
865 0.7 -6.8 -12 0 21 didnt 0
866 0.7 -7.09 -12 0 20 didnt 0
867 0.7 -7.033333333333333 -12 0 18 didnt 0
868 0.7 -7.1066666666666665 -12 0 18 didnt 0
869 0.7 -6.97 -12 0 20 didnt 0
870 0.7 -7.06 -12 0 21 didnt 0
871 0.7 -6.986666666666666 -12 0 28 didnt 0
872 0.7 -6.866666666666666 -12 0 27 didnt 0
873 0.7 -7.01 

1054 0.7 -7.446666666666666 -12 0 21 didnt 0
1055 0.7 -7.3533333333333335 -12 0 22 didnt 0
1056 0.7 -7.27 -12 0 19 didnt 0
1057 0.7 -7.203333333333333 -12 0 20 didnt 0
1058 0.7 -6.9366666666666665 -12 0 26 didnt 0
1059 0.7 -7.116666666666666 -12 0 26 didnt 0
1060 0.7 -7.03 -12 0 22 didnt 0
1061 0.7 -6.973333333333334 -12 0 20 didnt 0
1062 0.7 -6.983333333333333 -12 0 19 didnt 0
1063 0.7 -7.003333333333333 -12 0 22 didnt 0
1064 0.7 -6.6866666666666665 -12 0 22 didnt 0
1065 0.7 -6.706666666666667 -12 0 22 didnt 0
1066 0.7 -6.816666666666666 -12 0 22 didnt 0
1067 0.7 -7.0 -12 0 21 didnt 0
1068 0.7 -6.983333333333333 -12 0 22 didnt 0
1069 0.7 -7.243333333333333 -12 0 19 didnt 0
1070 0.7 -7.11 -12 0 21 didnt 0
1071 0.7 -6.92 -12 0 20 didnt 0
1072 0.7 -6.9 -12 0 24 didnt 0
1073 0.7 -7.03 -12 0 18 didnt 0
1074 0.7 -7.1866666666666665 -12 0 18 didnt 0
1075 0.7 -6.763333333333334 -12 0 24 didnt 0
1076 0.7 -6.913333333333333 -12 0 19 didnt 0
1077 0.7 -6.796666666666667 -12 0 24 didnt 0
1078 0.7 

1256 0.7 -7.033333333333333 -12 0 20 didnt 0
1257 0.7 -6.976666666666667 -12 0 20 didnt 0
1258 0.7 -7.206666666666667 -12 0 21 didnt 0
1259 0.7 -7.166666666666667 -12 0 19 didnt 0
1260 0.7 -7.066666666666666 -12 0 23 didnt 0
1261 0.7 -7.07 -12 0 21 didnt 0
1262 0.7 -6.71 -12 0 24 didnt 0
1263 0.7 -6.906666666666666 -12 0 21 didnt 0
1264 0.7 -6.816666666666666 -11 0 21 didnt 0
1265 0.7 -6.97 -12 0 21 didnt 0
1266 0.7 -6.736666666666666 -12 0 24 didnt 0
1267 0.7 -7.026666666666666 -12 0 20 didnt 0
1268 0.7 -6.843333333333334 -12 0 23 didnt 0
1269 0.7 -6.913333333333333 -12 0 18 didnt 0
1270 0.7 -6.773333333333333 -12 0 20 didnt 0
1271 0.7 -6.783333333333333 -12 0 20 didnt 0
1272 0.7 -6.963333333333333 -12 0 20 didnt 0
1273 0.7 -7.003333333333333 -12 0 21 didnt 0
1274 0.7 -6.86 -12 0 22 didnt 0
1275 0.7 -6.776666666666666 -12 0 22 didnt 0
1276 0.7 -6.81 -12 0 17 didnt 0
1277 0.7 -6.933333333333334 -12 0 20 didnt 0
1278 0.7 -7.1433333333333335 -12 0 19 didnt 0
1279 0.7 -7.1466666666666665 

1459 0.7 -7.096666666666667 -12 0 16 didnt 0
1460 0.7 -7.066666666666666 -12 0 22 didnt 0
1461 0.7 -6.786666666666667 -12 0 19 didnt 0
1462 0.7 -6.72 -12 0 22 didnt 0
1463 0.7 -6.836666666666667 -12 0 22 didnt 0
1464 0.7 -7.066666666666666 -12 0 24 didnt 0
1465 0.7 -7.036666666666667 -12 0 21 didnt 0
1466 0.7 -6.886666666666667 -12 0 21 didnt 0
1467 0.7 -7.316666666666666 -12 0 19 didnt 0
1468 0.7 -7.043333333333333 -12 0 17 didnt 0
1469 0.7 -7.076666666666667 -12 0 19 didnt 0
1470 0.7 -7.51 -12 0 16 didnt 0
1471 0.7 -7.21 -12 0 20 didnt 0
1472 0.7 -6.923333333333333 -12 0 16 didnt 0
1473 0.7 -6.956666666666667 -12 0 17 didnt 0


KeyboardInterrupt: 

In [ ]:
[[(1, 4), (2, 3), (5, 7), (6, 8), (3, 4), (6, 5), (8, 1), (2, 7), (7, 1), (3, 6), (8, 5), (4, 2), (3, 7), (5, 2), (4, 8), (1, 6), (3, 1), (7, 2), (6, 4), (5, 8), (2, 8), (4, 5), (7, 6), (1, 3), (2, 5), (3, 8), (4, 7), (6, 1), (4, 6), (8, 3), (1, 7), (7, 5), (4, 1), (8, 7), (2, 6), (5, 3), (8, 2), (3, 5), (7, 4), (1, 8), (6, 2), (8, 4), (7, 3), (5, 1), (6, 7), (1, 5), (4, 3), (2, 1), (1, 2), (5, 6), (7, 8), (2, 4), (5, 4), (3, 2), (8, 6), (6, 3)], [(5, 2), (7, 1), (4, 8), (3, 6), (4, 5), (3, 2), (6, 1), (8, 7), (8, 3), (6, 7), (1, 4), (2, 5), (7, 3), (8, 6), (2, 4), (1, 5), (6, 5), (4, 1), (3, 8), (2, 7), (5, 6), (1, 3), (7, 2), (8, 4), (5, 8), (4, 7), (2, 6), (3, 1), (2, 8), (5, 7), (3, 4), (1, 6), (5, 4), (7, 8), (6, 2), (2, 3), (4, 6), (1, 8), (3, 7), (6, 3), (8, 1), (7, 5), (4, 3), (3, 5), (2, 1), (7, 6), (8, 5), (1, 7), (1, 2), (5, 3), (6, 4), (5, 1), (8, 2), (7, 4), (6, 8), (4, 2)]]

In [560]:
unique=set()
for i in population:
    if nConflicts(i,nTeams)==0:
        unique.add(tuple(i))

In [561]:
len(unique)

2

In [549]:
for i in unique:
    print(i)
    for team in teamsInGames(list(i)):
        if teamsInGames(list(i)).count(team)!=nTeams:
            print(team,teamsInGames(list(i)).count(team),len(i))
    print()

((11, 5), (2, 14), (15, 7), (4, 3), (8, 13), (12, 10), (1, 16), (6, 9), (5, 4), (7, 14), (3, 12), (1, 6), (16, 9), (13, 11), (2, 8), (10, 15), (2, 5), (3, 8), (7, 15), (16, 13), (11, 4), (10, 14), (12, 9), (6, 1), (11, 16), (5, 3), (9, 13), (6, 4), (8, 12), (10, 7), (15, 14), (2, 1), (9, 5), (16, 8), (7, 10), (1, 15), (6, 2), (13, 3), (11, 12), (14, 4), (2, 12), (5, 7), (4, 10), (9, 14), (3, 1), (16, 6), (15, 8), (11, 13), (12, 1), (6, 8), (2, 16), (4, 14), (10, 3), (15, 13), (7, 5), (11, 9), (7, 3), (8, 10), (15, 9), (5, 12), (13, 6), (16, 2), (11, 14), (4, 1), (8, 11), (4, 7), (3, 15), (10, 12), (13, 9), (14, 1), (5, 16), (2, 6), (3, 9), (4, 8), (13, 14), (2, 7), (1, 12), (15, 10), (5, 11), (6, 16), (16, 5), (10, 4), (12, 6), (8, 2), (9, 1), (15, 11), (7, 13), (3, 14), (7, 11), (3, 10), (5, 8), (6, 14), (12, 16), (9, 2), (13, 4), (15, 1), (12, 5), (16, 10), (14, 13), (4, 15), (11, 3), (8, 1), (6, 7), (2, 9), (6, 10), (14, 16), (12, 4), (11, 7), (1, 2), (8, 9), (15, 3), (5, 13), (1, 1